In [10]:
import pandas as pd
import ast
import networkx as nx
import numpy as np
from tqdm import tqdm

In [11]:
df_CSS_papers = pd.read_csv('my_data/df_CSS_paper.csv')

In [14]:
author_list =  list(set([y for x in df_CSS_papers['authors'] for y in ast.literal_eval(x)]))
author_pairs = set()

for i, author in enumerate(author_list):
    for colab in author_list[i + 1:]:
        author_pairs.add((author, colab))
author_pairs = list(author_pairs)

In [ ]:
weighted_edges = []
for pair in tqdm(author_pairs):
    counter = 0
    for sublist in df_CSS_papers['authors']:
        sublist = ast.literal_eval(sublist)
        if pair[0] in sublist:
            if pair[1] in sublist:
                counter += 1
    weighted_edges.append((*pair, counter))

  3%|▎         | 10816/326028 [01:03<32:32, 161.45it/s]

In [ ]:
scc_graph = nx.Graph()
scc_graph.add_weighted_edges_from(weighted_edges)